# Time Series Analysis
## Data Gathering

In [3]:
import pandas as pd
import os

cwd = os.getcwd()

In [4]:
data_url = [r"\Model Data\X Train.csv", r"\Model Data\X Test.csv", r"\Model Data\Y Train.csv", r"\Model Data\Y Test.csv"]

In [21]:
dataframes = [pd.read_csv(cwd + url) for url in data_url]
data_values = dataframes[:2]
data_labels = dataframes[2:]

data = pd.concat(data_values, ignore_index=True, axis=0)
labels = pd.concat(data_labels, ignore_index=True, axis=0)

## Stationary Analysis

In [22]:
data.head()

,temperature,relative_humidity,wind_speed,wind_direction,total_precipitation,volumetric_soil_water_layer_1,Counties_Idx,Started_Date
0,9.53,0.80,2.31,289.30,0.00,0.24,16,2019-02-15
1,31.33,0.70,1.77,202.78,0.11,0.39,45,2018-06-28
2,23.03,0.48,3.34,136.20,0.00,0.08,44,2018-08-15
3,17.85,0.90,1.95,291.47,0.00,0.05,10,2019-07-12
4,26.57,0.44,4.81,76.10,0.00,0.10,2,2013-07-19
